# Read pathway  

In [2]:
import pandas as pd
import cobra
from collections import defaultdict
E0 = cobra.io.read_sbml_model("./models/iML1515_E0.xml")
def get_gene_id(model, gene_name):
    for i in model.genes:
        if(i.name == gene_name):
            return(i.id)
        
def convert_arg_to_list(arg):
    if type(arg) is not list and type(arg) is not tuple:
        arg = [arg]
    return(arg)

def record_rct(model, current_gene):
    return [rct.id for rct in model.genes.get_by_id(get_gene_id(model, current_gene)).reactions]

label_df = pd.read_excel(open('./Data/iML1515_GP.xlsx', 'rb'),
              sheet_name='iML1515_GP', index_col=0)

In [3]:
comp_df = pd.read_csv("./Data/alpha_table.csv")
genes = comp_df.Gene_inhibition[:]

In [4]:
genes_pwy_dict = defaultdict(list)
for current_gene in comp_df.Gene_inhibition[:]:
    genes_pwy_dict[f'{current_gene}'] = record_rct(E0, current_gene)

In [5]:
def get_pwy_dict():
    pwy_dict = defaultdict(list)
    for current_gene,rcts in genes_pwy_dict.items():
        for current_reaction in rcts:
            try:
    #             pwy_dict[f'{current_gene}'].append(set(label_df.loc[f'{current_reaction}', 'm_subsystem']))
                label = label_df.loc[f'{current_reaction}', 'm_subsystem']
                label = label if type(label) == str else label[0]
                pwy_dict[f'{current_gene}'].append(label)
            except:
                print(current_gene, f'reaction {current_reaction} not exist in list')
        pwy_dict[f'{current_gene}'] = set(convert_arg_to_list(pwy_dict[f'{current_gene}'])) | set(pwy_dict[f'{current_gene}'])
    return pwy_dict

pwy_dict = get_pwy_dict()

thrB reaction 4HTHRK not exist in list
glyA reaction THRA not exist in list
glyA reaction THRA2 not exist in list


In [6]:
pwy_rct_dict = defaultdict(list)
for index, sub_df in label_df[['m_subsystem']].iterrows():
#     print(index)
    pwy_rct_dict[sub_df[0]].append(index)


# pathway_ddict for comb

In [8]:
comb_list = list(pd.read_csv('./Data/gr_Div_DG_Blis_Aug31.csv').gene_inhibition[1:])
gene_comb_dict = dict()
# for gene_comb in comb_list:
gene_pathway_row = list()
for gene_comb in comb_list:
    gene_pair = gene_comb.split('.')
    gene_comb_dict[gene_comb] = pwy_dict[gene_pair[0]] | pwy_dict[gene_pair[1]] 

#     gene_pathway_row = [gene_comb]

In [9]:
def pwy_dict_to_df(pwy_dict):
    df = pd.DataFrame.from_dict({k: ' + '.join(v) for k, v in pwy_dict.items()},
#     df = pd.DataFrame.from_dict({k: '+'.join(v) for k, v in pwy_dict.items()},
                       orient='index', columns=['Pathway'])
    df.index.name = 'Gene_inhibition'
    return df
genes_combo_pathway = pwy_dict_to_df(gene_comb_dict)
genes_combo_pathway.to_csv('./Data/gene_combo_pathway.csv')

# convert_dict_to_long_df

In [14]:
full_dict = pwy_dict.copy()
full_dict.update(gene_comb_dict)

In [50]:
def convert_dict_to_long_df(d: dict, cols: list):
    gcomb_pwy_list = list()
    for gcomb, pwys in d.items():
        for pwy in pwys:
            gcomb_pwy_list.append([gcomb, pwy])
#     index_col = [col for col in cols if col in 'Gene_inhibition|Reaction']
    df = pd.DataFrame.from_records(gcomb_pwy_list, columns=cols)
    return df
rct_pathway_df = convert_dict_to_long_df(pwy_rct_dict,['Pathway','Reaction'])
rct_pathway_df.set_index('Reaction').to_csv('./Data/rct_pathway.csv')

In [518]:
DG


,Pathway,XG,Gene_list,Gene_count,Antagonistic Ratio
0,Alanine and Aspartate Metabolism,DG,"dadX.rffG, dadX.pyrD, dadX.guaB, dadX.acnB, da...",10,0.300000
2,Alternate Carbon Metabolism,DG,"dadX.acnB, acnB.thrB, dapF.acnB, acnB.gltA, ac...",6,0.166667
4,Arginine and Proline Metabolism,DG,aroA.argD,1,1.000000
6,Cell Envelope Biosynthesis,DG,"dadX.rffG, murA.mrdA, murA.pyrE, rffG.pyrD, mu...",8,0.125000
8,Citric Acid Cycle,DG,"gltA.gltD, dadX.acnB, acnB.thrB, dapF.acnB, ac...",7,0.142857
10,Cofactor and Prosthetic Group Biosynthesis,DG,"dapA.serC, serC.dapD, serC.dapB, aroA.serC, se...",13,0.000000
13,Glutamate Metabolism,DG,gltA.gltD,1,0.000000
15,Glycine and Serine Metabolism,DG,"dapA.serC, serC.dapD, serC.dapB, aroA.serC, se...",5,0.000000
17,Glycolysis/Gluconeogenesis,DG,"dadX.eno, dadX.gapA",2,0.000000
20,Metabolite Repair,DG,dadX.gapA,1,0.000000


In [523]:
DG['Antagonistic_gcomb'] = DG.Gene_list.apply(lambda x: antagonistic_count(x.split(', ')))
DG.Antagonistic_gcomb./DG.Gene_count)

TypeError: ufunc 'true_divide' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [539]:
gene_count_df

,Pathway,XG,Gene_list,Gene_count
0,Alanine and Aspartate Metabolism,DG,"dadX.rffG, dadX.pyrD, dadX.guaB, dadX.acnB, da...",10
1,Alanine and Aspartate Metabolism,SG,dadX,1
2,Alternate Carbon Metabolism,DG,"dadX.acnB, acnB.thrB, dapF.acnB, acnB.gltA, ac...",6
3,Alternate Carbon Metabolism,SG,"acnB, pfkA",2
4,Arginine and Proline Metabolism,DG,aroA.argD,1
5,Arginine and Proline Metabolism,SG,argD,1
6,Cell Envelope Biosynthesis,DG,"dadX.rffG, murA.mrdA, murA.pyrE, rffG.pyrD, mu...",8
7,Cell Envelope Biosynthesis,SG,"murA, rffG",2
8,Citric Acid Cycle,DG,"gltA.gltD, dadX.acnB, acnB.thrB, dapF.acnB, ac...",7
9,Citric Acid Cycle,SG,"gltA, acnB",2


In [540]:
gene_count_df.merge(get_ratio_col(gene_count_df), on='Pathway')

KeyError: "['Antagonistic Ratio'] not in index"

In [546]:
    gene_count_df = (gene_pathway_df.groupby(['Pathway','XG'], as_index=False).agg(
                        Gene_list=('Gene_inhibition', lambda x: ', '.join(x.values)),
                        Gene_count=('Gene_inhibition', 'count')))


In [548]:
get_ratio_col(gene_count_df)

,Pathway,Antagonistic_ratio,Antagonistic_gcomb
0,Alanine and Aspartate Metabolism,0.300000,"dadX.guaB, dadX.rffG, dadX.pyrD"
2,Alternate Carbon Metabolism,0.166667,acnB.thrB
4,Arginine and Proline Metabolism,1.000000,aroA.argD
6,Cell Envelope Biosynthesis,0.125000,dadX.rffG
8,Citric Acid Cycle,0.142857,acnB.thrB
10,Cofactor and Prosthetic Group Biosynthesis,0.076923,
13,Glutamate Metabolism,1.000000,
15,Glycine and Serine Metabolism,0.200000,
17,Glycolysis/Gluconeogenesis,0.500000,
20,Metabolite Repair,1.000000,


In [554]:
DG.Antagonistic_gcomb.tolist()

['dadX.guaB, dadX.rffG, dadX.pyrD',
 'acnB.thrB',
 'aroA.argD',
 'dadX.rffG',
 'acnB.thrB',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'aroA.guaB, dadX.guaB, dadX.pyrD',
 '',
 'aroA.argD, acnB.thrB, aroA.dapB',
 'aroA.guaB, aroA.argD, aroA.dapB']

In [587]:
gene_count_df.query('XG=="DG"').copy()

,Pathway,XG,Gene_list,Gene_count
0,Alanine and Aspartate Metabolism,DG,"dadX.rffG, dadX.pyrD, dadX.guaB, dadX.acnB, da...",10
1,Alternate Carbon Metabolism,DG,"dadX.acnB, acnB.thrB, dapF.acnB, acnB.gltA, ac...",6
2,Arginine and Proline Metabolism,DG,aroA.argD,1
3,Cell Envelope Biosynthesis,DG,"dadX.rffG, murA.mrdA, murA.pyrE, rffG.pyrD, mu...",8
4,Citric Acid Cycle,DG,"gltA.gltD, dadX.acnB, acnB.thrB, dapF.acnB, ac...",7
5,Cofactor and Prosthetic Group Biosynthesis,DG,"dapA.serC, serC.dapD, serC.dapB, aroA.serC, se...",13
6,Glutamate Metabolism,DG,gltA.gltD,1
7,Glycine and Serine Metabolism,DG,"dapA.serC, serC.dapD, serC.dapB, aroA.serC, se...",5
8,Glycolysis/Gluconeogenesis,DG,"dadX.eno, dadX.gapA",2
9,Metabolite Repair,DG,dadX.gapA,1


In [585]:
DG = gene_count_df.query('XG=="DG"').copy().reindex()
DG['Antagonistic_gcomb'] = DG.Gene_list.apply(lambda x: antagonistic_count(x.split(', ')))
DG['Antagonistic_ratio'] =pd.Series(map(len, DG.Antagonistic_gcomb.tolist()))

In [584]:
pd.Series(map(len, DG.Antagonistic_gcomb.tolist()))

0     3
1     1
2     1
3     1
4     1
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    3
13    0
14    3
15    3
dtype: int64

In [583]:
DG

,Pathway,XG,Gene_list,Gene_count,Antagonistic_gcomb,Antagonistic_ratio
0,Alanine and Aspartate Metabolism,DG,"dadX.rffG, dadX.pyrD, dadX.guaB, dadX.acnB, da...",10,"[dadX.guaB, dadX.rffG, dadX.pyrD]",3.0
2,Alternate Carbon Metabolism,DG,"dadX.acnB, acnB.thrB, dapF.acnB, acnB.gltA, ac...",6,[acnB.thrB],1.0
4,Arginine and Proline Metabolism,DG,aroA.argD,1,[aroA.argD],1.0
6,Cell Envelope Biosynthesis,DG,"dadX.rffG, murA.mrdA, murA.pyrE, rffG.pyrD, mu...",8,[dadX.rffG],0.0
8,Citric Acid Cycle,DG,"gltA.gltD, dadX.acnB, acnB.thrB, dapF.acnB, ac...",7,[acnB.thrB],0.0
10,Cofactor and Prosthetic Group Biosynthesis,DG,"dapA.serC, serC.dapD, serC.dapB, aroA.serC, se...",13,[],0.0
13,Glutamate Metabolism,DG,gltA.gltD,1,[],0.0
15,Glycine and Serine Metabolism,DG,"dapA.serC, serC.dapD, serC.dapB, aroA.serC, se...",5,[],3.0
17,Glycolysis/Gluconeogenesis,DG,"dadX.eno, dadX.gapA",2,[],NaN
20,Metabolite Repair,DG,dadX.gapA,1,[],NaN


In [596]:
# DG['Antagonistic_ratio'] =pd.Series(map(len, DG.Antagonistic_gcomb.tolist()))
# .map(len)
DG['Antagonistic_gcomb'].apply(lambda x: ', '.join(x))

0     dadX.guaB, dadX.rffG, dadX.pyrD
2                           acnB.thrB
4                           aroA.argD
6                           dadX.rffG
8                           acnB.thrB
10                                   
13                                   
15                                   
17                                   
20                                   
22                                   
24                                   
26    aroA.guaB, dadX.guaB, dadX.pyrD
28                                   
30    aroA.argD, acnB.thrB, aroA.dapB
32    aroA.guaB, aroA.argD, aroA.dapB
Name: Antagonistic_gcomb, dtype: object

In [598]:
antagonistic_set = ['dadX.rffG', 'dadX.pyrD', 'acnB.thrB', 'acnB.thrB', 'acnB.thrB',
       'aroA.argD', 'aroA.argD', 'aroA.argD', 'dadX.guaB', 'aroA.dapB',
       'aroA.guaB']

def get_gene_pathway_df(full_dict):
    gene_pathway_df = convert_dict_to_long_df(full_dict, ['Gene_inhibition', 'Pathway'])
    gene_pathway_df['XG'] = gene_pathway_df.Gene_inhibition.str.split('.').apply(
        lambda x: 'SG' if len(x)==1 else 'DG') 
    gene_pathway_df.set_index('Gene_inhibition').to_csv('./Data/gene_pathway.csv')
    return gene_pathway_df

def antagonistic_count(genes:list):
    return list((set(genes).intersection(antagonistic_set)))

def get_ratio_col(gene_count_df):
    DG = gene_count_df.query('XG=="DG"').reset_index(drop=True).copy()
    DG['Antagonistic_gcomb'] = DG.Gene_list.apply(lambda x: antagonistic_count(x.split(', ')))
#     DG['Antagonistic_ratio'] = DG.Antagonistic_gcomb.str.split(', ').map(len)
    DG['Antagonistic_ratio'] = pd.Series(map(len, DG.Antagonistic_gcomb.tolist()))/DG.Gene_count
    DG['Antagonistic_gcomb'] = DG['Antagonistic_gcomb'].apply(lambda x: ', '.join(x))
    
    return DG[['Pathway', 'Antagonistic_ratio', 'Antagonistic_gcomb']]

def get_gene_count_df(gene_pathway_df):
    gene_count_df = (gene_pathway_df.groupby(['Pathway','XG'], as_index=False).agg(
                        Gene_list=('Gene_inhibition', lambda x: ', '.join(x.values)),
                        Gene_count=('Gene_inhibition', 'count')))
    gene_count_df = gene_count_df.merge(get_ratio_col(gene_count_df), on='Pathway')
    (gene_count_df.pivot(index=['Pathway','Antagonistic_ratio', 'Antagonistic_gcomb'], columns=['XG'], values=['Gene_list','Gene_count'])
         .sort_values(by=('Gene_count','DG'), ascending=False)).to_csv('./Data/gene_count.csv')
    return gene_count_df

def get_single_pathway_df(gene_pathway_df):
    # add p_o afterwards
    df = (gene_pathway_df.groupby(['Gene_inhibition','XG'], as_index=False).agg(
        Pathway_count=('Pathway', 'count'),
        Pathway_list = ('Pathway', lambda x: list(x.values))))
#         P = ('Pathway', lambda x: pd.DataFrame(x.values))))
    df['Single_pathway'] = pd.cut(df['Pathway_count'], [0,1,10], labels = ['Single_pathway', 'Multi_pathway'])
    df.to_csv('./Data/single_pathway_df.csv', index=False)
    return df.sort_values(['Pathway_count', 'XG'], ascending=True)

In [599]:
gene_pathway_df = get_gene_pathway_df(full_dict)
gene_count_df=get_gene_count_df(gene_pathway_df)

# single pathway
single_pathway_df = get_single_pathway_df(gene_pathway_df)

In [615]:
single_pathway_df = get_single_pathway_df(gene_pathway_df)

In [614]:
single_pathway_df

,Gene_inhibition,XG,Pathway_count,Pathway_list,Single_pathway
13,aroA.pheA,DG,1,"[Tyrosine, Tryptophan, and Phenylalanine Metab...",Single_pathway
28,dapA.dapF,DG,1,[Threonine and Lysine Metabolism],Single_pathway
34,dapF.dapD,DG,1,[Threonine and Lysine Metabolism],Single_pathway
42,folC.folA,DG,1,[Cofactor and Prosthetic Group Biosynthesis],Single_pathway
45,folP.folA,DG,1,[Cofactor and Prosthetic Group Biosynthesis],Single_pathway
...,...,...,...,...,...
68,pfkA,SG,3,"[Alternate Carbon Metabolism, Glycolysis/Gluco...",Multi_pathway
84,serC,SG,3,"[Glycine and Serine Metabolism, Cofactor and P...",Multi_pathway
5,acnB.purT,DG,4,"[Alternate Carbon Metabolism, Pyruvate Metabol...",Multi_pathway
14,aroA.serC,DG,4,"[Threonine and Lysine Metabolism, Glycine and ...",Multi_pathway


# p_o

In [434]:
def get_col_p_o(gr_df):
    col_p_o = gr_df.Predicted_additive_effect_E0_coculture-gr_df.E0_coculture
    col_p_o[(gr_df.Predicted_additive_effect_E0_coculture < 1.5e-8) & (gr_df.E0_coculture< 1.5e-8)] = 0
    return pd.DataFrame(col_p_o, columns=['P_O'])

def get_antagonistic_df(syn_df):
    antagonistic_list = syn_df.loc[syn_df['P_O']<0].query("P_O<-0.01").index 
    # ? func get pwy col
    potential_pwy = (gene_combo_pathway
                                      .loc[antagonistic_list,'Pathway']
                                      .str.split(' \+ ') # series string need \+
                                      .apply(lambda x: sorted(x))) 
    return pd.DataFrame(potential_pwy)

In [436]:
gene_combo_pathway =  pd.read_csv('./Data/gene_combo_pathway.csv', index_col='Gene_inhibition')
gr_DG = pd.read_csv('./r/data fitting/standardized_gr_Div_DG_Mult_Aug31.csv', index_col='gene_inhibition')


In [437]:
syn_df = get_col_p_o(gr_DG)
antagonistic_df = get_antagonistic_df(syn_df)



#  Pathway 1234

In [258]:
df = gene_pathway_df.sort_values(['Pathway'], ascending=True)
df
df['Counter'] = df.groupby(['Gene_inhibition','XG'], as_index=False).cumcount()+1
df['Counter'] = df.Counter.apply(lambda x: ''.join(['Pathway_No_', str(x)]))

In [372]:
single_pathway = find_single_pathway(gene_pathway_df)
# single_pathway.query('Single_pathway=="Single_pathway" & XG=="DG"').set_index('Gene_inhibition')[['Pathway_list']]

,Pathway_list
Gene_inhibition,
aroA.pheA,"[Tyrosine, Tryptophan, and Phenylalanine Metab..."
dapA.dapF,[Threonine and Lysine Metabolism]
dapF.dapD,[Threonine and Lysine Metabolism]
folC.folA,[Cofactor and Prosthetic Group Biosynthesis]
folP.folA,[Cofactor and Prosthetic Group Biosynthesis]
folP.folC,[Cofactor and Prosthetic Group Biosynthesis]
mrdA.mrcA,[Murein Biosynthesis]
pyrE.pyrD,[Purine and Pyrimidine Biosynthesis]
thrC.thrB,[Threonine and Lysine Metabolism]


In [241]:
df.loc[df.Gene_inhibition=='dadX.gapA',:]

,Gene_inhibition,Pathway,XG,Counter
154,dadX.gapA,Metabolite Repair,DG,Pathway No. 1
155,dadX.gapA,Glycolysis/Gluconeogenesis,DG,Pathway No. 2
156,dadX.gapA,Alanine and Aspartate Metabolism,DG,Pathway No. 3
157,dadX.gapA,Cofactor and Prosthetic Group Biosynthesis,DG,Pathway No. 4


In [354]:
gene_pathway_df

,Gene_inhibition,Pathway,XG
0,gapA,Metabolite Repair,SG
1,gapA,Cofactor and Prosthetic Group Biosynthesis,SG
2,gapA,Glycolysis/Gluconeogenesis,SG
3,gltA,Citric Acid Cycle,SG
4,glnA,Glutamate Metabolism,SG
...,...,...,...
157,dadX.gapA,Cofactor and Prosthetic Group Biosynthesis,DG
158,acnB.purT,Alternate Carbon Metabolism,DG
159,acnB.purT,Pyruvate Metabolism,DG
160,acnB.purT,Purine and Pyrimidine Biosynthesis,DG


In [243]:
df.sort_values(['Gene_inhibition','Counter'], ascending=True).set_index(['Gene_inhibition','Counter'])

Pathway  \
Gene_inhibition Counter                                                            
aceF            Pathway No. 1                         Glycolysis/Gluconeogenesis   
ackA            Pathway No. 1                                Pyruvate Metabolism   
acnB            Pathway No. 1                                  Citric Acid Cycle   
                Pathway No. 2                        Alternate Carbon Metabolism   
acnB.ackA       Pathway No. 1                        Alternate Carbon Metabolism   
...                                                                          ...   
tktA            Pathway No. 1                          Pentose Phosphate Pathway   
tktA.gnd        Pathway No. 1                          Pentose Phosphate Pathway   
trpA            Pathway No. 1  Tyrosine, Tryptophan, and Phenylalanine Metabo...   
trpA.aroA       Pathway No. 1  Tyrosine, Tryptophan, and Phenylalanine Metabo...   
yrbG            Pathway No. 1             Inorganic Ion Transport and Metabolism   

                               XG  
Gene_inhibition Counter            
aceF            Pathway No. 1  SG  
ackA            Pathway No. 1  SG  
acnB            Pathway No. 1  SG  
                Pathway No. 2  SG  
acnB.ackA       Pathway No. 1  DG  
...                            ..  
tktA            Pathway No. 1  SG  
tktA.gnd        Pathway No. 1  DG  
trpA            Pathway No. 1  SG  
trpA.aroA       Pathway No. 1  DG  
yrbG            Pathway No. 1  SG  

[162 rows x 2 columns]

In [236]:
len(df.Gene_inhibition.unique())

97

In [246]:
df.columns

Index(['Gene_inhibition', 'Pathway', 'XG', 'Counter'], dtype='object')

In [306]:
# a = df.pivot(index=['Gene_inhibition','XG'],columns='Counter', values=['Pathway'])
a = df.pivot(index=['Gene_inhibition','XG'],columns='Counter', values=['Pathway']).Pathway.reset_index()

In [333]:
df.pivot(index=['Gene_inhibition','XG'],columns='Counter', values=['Pathway']).unstack(level=1)

Pathway  \
Counter                                               Pathway_No_1   
XG                                                              DG   
Gene_inhibition                                                      
aceF                                                           NaN   
ackA                                                           NaN   
acnB                                                           NaN   
acnB.ackA                              Alternate Carbon Metabolism   
acnB.gltA                              Alternate Carbon Metabolism   
...                                                            ...   
tktA                                                           NaN   
tktA.gnd                                 Pentose Phosphate Pathway   
trpA                                                           NaN   
trpA.aroA        Tyrosine, Tryptophan, and Phenylalanine Metabo...   
yrbG                                                           NaN   

                                                                    \
Counter                                                              
XG                                                              SG   
Gene_inhibition                                                      
aceF                                    Glycolysis/Gluconeogenesis   
ackA                                           Pyruvate Metabolism   
acnB                                   Alternate Carbon Metabolism   
acnB.ackA                                                      NaN   
acnB.gltA                                                      NaN   
...                                                            ...   
tktA                                     Pentose Phosphate Pathway   
tktA.gnd                                                       NaN   
trpA             Tyrosine, Tryptophan, and Phenylalanine Metabo...   
trpA.aroA                                                      NaN   
yrbG                        Inorganic Ion Transport and Metabolism   

                                                                            \
Counter               Pathway_No_2                            Pathway_No_3   
XG                              DG                 SG                   DG   
Gene_inhibition                                                              
aceF                           NaN                NaN                  NaN   
ackA                           NaN                NaN                  NaN   
acnB                           NaN  Citric Acid Cycle                  NaN   
acnB.ackA        Citric Acid Cycle                NaN  Pyruvate Metabolism   
acnB.gltA        Citric Acid Cycle                NaN                  NaN   
...                            ...                ...                  ...   
tktA                           NaN                NaN                  NaN   
tktA.gnd                       NaN                NaN                  NaN   
trpA                           NaN                NaN                  NaN   
trpA.aroA                      NaN                NaN                  NaN   
yrbG                           NaN                NaN                  NaN   

                                        
Counter              Pathway_No_4       
XG                SG           DG   SG  
Gene_inhibition                         
aceF             NaN          NaN  NaN  
ackA             NaN          NaN  NaN  
acnB             NaN          NaN  NaN  
acnB.ackA        NaN          NaN  NaN  
acnB.gltA        NaN          NaN  NaN  
...              ...          ...  ...  
tktA             NaN          NaN  NaN  
tktA.gnd         NaN          NaN  NaN  
trpA             NaN          NaN  NaN  
trpA.aroA        NaN          NaN  NaN  
yrbG             NaN          NaN  NaN  

[97 rows x 8 columns]

In [335]:
df.set_index('XG').unstack()

                 XG
Gene_inhibition  DG       dadX.mrcA
                 DG       dadX.acnB
                 DG       dadX.pheA
                 DG       dadX.mrdA
                 DG       dadX.aroA
                           ...     
Counter          DG    Pathway_No_2
                 DG    Pathway_No_2
                 DG    Pathway_No_2
                 DG    Pathway_No_1
                 DG    Pathway_No_2
Length: 486, dtype: object

In [328]:
a.pivot_table(index=['Pathway_No_1', 'Pathway_No_2', 'Pathway_No_3', 'Pathway_No_4'],
        columns='XG', values=['Gene_inhibition'], )

D:\anaconda3\envs\vr\lib\site-packages\pandas\core\apply.py:507: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  return self._try_aggregate_string_function(obj, f, *self.args, **self.kwargs)


Pathway_No_1,Pathway_No_2,Pathway_No_3,Pathway_No_4
Alanine and Aspartate Metabolism,Cofactor and Prosthetic Group Biosynthesis,Glycolysis/Gluconeogenesis,Metabolite Repair
Alternate Carbon Metabolism,Citric Acid Cycle,Purine and Pyrimidine Biosynthesis,Pyruvate Metabolism
Cofactor and Prosthetic Group Biosynthesis,Glycine and Serine Metabolism,Threonine and Lysine Metabolism,"Tyrosine, Tryptophan, and Phenylalanine Metabolism"


In [314]:
[col for col in a.columns if 'Pathway' in col]

['Pathway_No_1', 'Pathway_No_2', 'Pathway_No_3', 'Pathway_No_4']

In [326]:
df.pivot_table(columns='XG', values=['Gene_inhibition'])
# .set_index(['SG','DG'])

D:\anaconda3\envs\vr\lib\site-packages\pandas\core\apply.py:507: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  return self._try_aggregate_string_function(obj, f, *self.args, **self.kwargs)


XG


Counter,Gene_inhibition,XG,Pathway_No_1,Pathway_No_2,Pathway_No_3,Pathway_No_4
0,aceF,SG,Glycolysis/Gluconeogenesis,NaN,NaN,NaN
1,ackA,SG,Pyruvate Metabolism,NaN,NaN,NaN
2,acnB,SG,Alternate Carbon Metabolism,Citric Acid Cycle,NaN,NaN
3,acnB.ackA,DG,Alternate Carbon Metabolism,Citric Acid Cycle,Pyruvate Metabolism,NaN
4,acnB.gltA,DG,Alternate Carbon Metabolism,Citric Acid Cycle,NaN,NaN
...,...,...,...,...,...,...
92,tktA,SG,Pentose Phosphate Pathway,NaN,NaN,NaN
93,tktA.gnd,DG,Pentose Phosphate Pathway,NaN,NaN,NaN
94,trpA,SG,"Tyrosine, Tryptophan, and Phenylalanine Metabo...",NaN,NaN,NaN
95,trpA.aroA,DG,"Tyrosine, Tryptophan, and Phenylalanine Metabo...",NaN,NaN,NaN


In [320]:
a.reindex().pivot(columns='XG', values=['Gene_inhibition'])

Gene_inhibition      
XG              DG    SG
0              NaN  aceF
1              NaN  ackA
2              NaN  acnB
3        acnB.ackA   NaN
4        acnB.gltA   NaN
..             ...   ...
92             NaN  tktA
93        tktA.gnd   NaN
94             NaN  trpA
95       trpA.aroA   NaN
96             NaN  yrbG

[97 rows x 2 columns]

In [266]:
a.Pathway.Pathway_No_4.notna()

3

In [213]:
# pwy_count_df = 
df = find_single_pathway(gene_pathway_df)
df[['Gene_inhibition', 'Pathway_list']].

,Gene_inhibition,XG,Pathway_count,P,Single_pathway
0,aceF,SG,1,[Glycolysis/Gluconeogenesis],True
1,ackA,SG,1,[Pyruvate Metabolism],True
2,acnB,SG,2,"[Alternate Carbon Metabolism, Citric Acid Cycle]",False
3,acnB.ackA,DG,3,"[Alternate Carbon Metabolism, Pyruvate Metabol...",False
4,acnB.gltA,DG,2,"[Alternate Carbon Metabolism, Citric Acid Cycle]",False
...,...,...,...,...,...
92,tktA,SG,1,[Pentose Phosphate Pathway],True
93,tktA.gnd,DG,1,[Pentose Phosphate Pathway],True
94,trpA,SG,1,"[Tyrosine, Tryptophan, and Phenylalanine Metab...",True
95,trpA.aroA,DG,1,"[Tyrosine, Tryptophan, and Phenylalanine Metab...",True


In [181]:
pwy_count_df

,Gene_inhibition,XG,Pathway_count,Single_pathway
0,aceF,SG,1,True
1,ackA,SG,1,True
2,acnB,SG,2,False
3,acnB.ackA,DG,3,False
4,acnB.gltA,DG,2,False
...,...,...,...,...
92,tktA,SG,1,True
93,tktA.gnd,DG,1,True
94,trpA,SG,1,True
95,trpA.aroA,DG,1,True


In [165]:
gene_pathway_df

,Gene_inhibition,Pathway,XG
0,gapA,Metabolite Repair,SG
1,gapA,Cofactor and Prosthetic Group Biosynthesis,SG
2,gapA,Glycolysis/Gluconeogenesis,SG
3,gltA,Citric Acid Cycle,SG
4,glnA,Glutamate Metabolism,SG
...,...,...,...
157,dadX.gapA,Cofactor and Prosthetic Group Biosynthesis,DG
158,acnB.purT,Alternate Carbon Metabolism,DG
159,acnB.purT,Pyruvate Metabolism,DG
160,acnB.purT,Purine and Pyrimidine Biosynthesis,DG


In [ ]:
def get_sum_row(gene_comb_pathway_wide):
    XG_str = 'DG' if len(gene_comb_pathway_wide.index[0].split('.'))>1 else 'SG'
    sum_row = remove_Zero_col(
                pd.DataFrame(gene_comb_pathway_wide.sum(), columns=[f'{XG_str}_count']).T)
    pwy_ele = dict()
    df = remove_Zero_col(gene_comb_pathway_wide)
    for col in df:    
        pwy_ele[col]  = ', '.join(df.loc[df[f'{col}']==True, f'{col}'].index)
    pwy_ele_df = pd.DataFrame(pwy_ele,index=[XG_str])
    
    return pd.concat([pwy_ele_df, sum_row])


In [ ]:
def pathway_bool(pathway_query):
    return genes_pathway['Pathway'].apply(lambda x: pathway_query in x)
# pathway_bool('Glycolysis')

In [ ]:
df = sim.fluxes_by_species['E0']
df[df.columns[(df != 0).any(axis=0)]]
df[df.columns[(df != 0).any(axis=0)]].to_csv('flux.csv')

# E0 Carbon

In [608]:
exchange_rct = [rct.id for rct in E0.reactions if 'EX_' in rct.id]

In [603]:
CAC_rct = rct_pathway_df.query('Pathway=="Citric Acid Cycle"').Reaction.unique()

In [612]:
http://localhost:8888/notebooks/OneDrive/COMETS_Jupyter/Run_Comets/results_display.ipynb#Ac-flux-comparison

,Gene_inhibition,Pathway,XG
0,gapA,Metabolite Repair,SG
1,gapA,Cofactor and Prosthetic Group Biosynthesis,SG
2,gapA,Glycolysis/Gluconeogenesis,SG
3,gltA,Citric Acid Cycle,SG
4,glnA,Glutamate Metabolism,SG
...,...,...,...
157,dadX.gapA,Cofactor and Prosthetic Group Biosynthesis,DG
158,acnB.purT,Alternate Carbon Metabolism,DG
159,acnB.purT,Pyruvate Metabolism,DG
160,acnB.purT,Purine and Pyrimidine Biosynthesis,DG
